# Notebook resposável por apresentar os componentes básicos do aggregate no mongo para realizar joins

In [1]:
from pymongo import MongoClient
from bson.son import SON
import pprint
import numpy as np
import pandas as pd

# para geração de dados fake
from faker import Faker
from pandasql import sqldf

## povoador

In [2]:
def criar_pessoa():
    primeiro_nome = fake.first_name()
    ultimo_nome = fake.last_name()
    
    idade = np.random.randint(10, 30)
    
    pessoa = {
        'cpf': fake.cpf(),
        'nome': f'{primeiro_nome} {ultimo_nome}' ,
        'idade': idade ,
    }
    
    primeiro_nome = primeiro_nome.replace(" ", "")
    ultimo_nome = ultimo_nome.replace(" ", "")
    
    pessoa['email']= f'{primeiro_nome.lower()}.{ultimo_nome.lower()}@{fake.free_email_domain()}'
    
    return pessoa

In [3]:
def criar_automoveis(pessoa):
    marcas = ['audi', 'bmw', 'citroen', 'ferrari', 'fiat', 'ford', 'jeep', 'mercedez', 'peugeut',
              'porsche', 'renault', 'suzuki', 'toyota', 'vw'] 
        
    automoveis = []
    n = np.random.randint(0, 5)
    
    for i in range(n):
        marca = np.random.choice(marcas, 1)[0]
        ano = np.random.randint(1970, 2022)
        dono_cpf = pessoa['cpf']
    
    
        automovel = {
            'marca': marca,
            'ano': ano,
            'dono_cpf': dono_cpf,
        }    
        automoveis.append(automovel)
    
    return automoveis

## Configurações

In [4]:
pp = pprint.PrettyPrinter(compact=True)
# gerando dados fictícios em português do Brasil
fake = Faker(['pt_BR'])

In [5]:
client = MongoClient('localhost', 27017)
# Se a base de dados não existir, ela é criada automagicamente :-)
db = client['techshot_aggregate_join']

In [6]:
lista_pessoas = []

for i in range(30):
    lista_pessoas.append(criar_pessoa())
    
pp.pprint(lista_pessoas)

[{'cpf': '498.137.502-60',
  'email': 'anaclara.moraes@gmail.com',
  'idade': 23,
  'nome': 'Ana Clara Moraes'},
 {'cpf': '405.912.638-15',
  'email': 'bruno.vieira@yahoo.com.br',
  'idade': 21,
  'nome': 'Bruno Vieira'},
 {'cpf': '509.634.721-43',
  'email': 'francisco.dacunha@ig.com.br',
  'idade': 20,
  'nome': 'Francisco da Cunha'},
 {'cpf': '738.594.026-92',
  'email': 'marcelo.almeida@bol.com.br',
  'idade': 24,
  'nome': 'Marcelo Almeida'},
 {'cpf': '369.785.041-57',
  'email': 'samuel.nunes@yahoo.com.br',
  'idade': 20,
  'nome': 'Samuel Nunes'},
 {'cpf': '184.930.576-57',
  'email': 'esther.sales@yahoo.com.br',
  'idade': 24,
  'nome': 'Esther Sales'},
 {'cpf': '742.853.691-09',
  'email': 'otávio.pires@yahoo.com.br',
  'idade': 18,
  'nome': 'Otávio Pires'},
 {'cpf': '826.950.173-59',
  'email': 'sophia.costa@gmail.com',
  'idade': 14,
  'nome': 'Sophia Costa'},
 {'cpf': '780.952.341-41',
  'email': 'brenda.nascimento@uol.com.br',
  'idade': 13,
  'nome': 'Brenda Nascimento'}

In [7]:
# cria a lista de veículos para as pessoas

lista_automoveis = []
for pessoa in lista_pessoas:
    lista_automoveis.extend(criar_automoveis(pessoa))
    
lista_automoveis    

[{'marca': 'audi', 'ano': 2021, 'dono_cpf': '498.137.502-60'},
 {'marca': 'citroen', 'ano': 2018, 'dono_cpf': '498.137.502-60'},
 {'marca': 'ford', 'ano': 1987, 'dono_cpf': '498.137.502-60'},
 {'marca': 'audi', 'ano': 2002, 'dono_cpf': '405.912.638-15'},
 {'marca': 'porsche', 'ano': 1979, 'dono_cpf': '405.912.638-15'},
 {'marca': 'ford', 'ano': 1983, 'dono_cpf': '405.912.638-15'},
 {'marca': 'audi', 'ano': 1998, 'dono_cpf': '405.912.638-15'},
 {'marca': 'audi', 'ano': 2013, 'dono_cpf': '509.634.721-43'},
 {'marca': 'mercedez', 'ano': 1979, 'dono_cpf': '509.634.721-43'},
 {'marca': 'audi', 'ano': 1979, 'dono_cpf': '509.634.721-43'},
 {'marca': 'ford', 'ano': 2003, 'dono_cpf': '509.634.721-43'},
 {'marca': 'peugeut', 'ano': 2018, 'dono_cpf': '738.594.026-92'},
 {'marca': 'renault', 'ano': 1978, 'dono_cpf': '738.594.026-92'},
 {'marca': 'vw', 'ano': 2011, 'dono_cpf': '369.785.041-57'},
 {'marca': 'peugeut', 'ano': 1996, 'dono_cpf': '369.785.041-57'},
 {'marca': 'audi', 'ano': 1970, 'dono_

In [8]:
pessoas = pd.DataFrame.from_records(lista_pessoas)
pessoas

,cpf,nome,idade,email
0,498.137.502-60,Ana Clara Moraes,23,anaclara.moraes@gmail.com
1,405.912.638-15,Bruno Vieira,21,bruno.vieira@yahoo.com.br
2,509.634.721-43,Francisco da Cunha,20,francisco.dacunha@ig.com.br
3,738.594.026-92,Marcelo Almeida,24,marcelo.almeida@bol.com.br
4,369.785.041-57,Samuel Nunes,20,samuel.nunes@yahoo.com.br
5,184.930.576-57,Esther Sales,24,esther.sales@yahoo.com.br
6,742.853.691-09,Otávio Pires,18,otávio.pires@yahoo.com.br
7,826.950.173-59,Sophia Costa,14,sophia.costa@gmail.com
8,780.952.341-41,Brenda Nascimento,13,brenda.nascimento@uol.com.br
9,724.560.193-34,Igor da Luz,21,igor.daluz@ig.com.br


In [9]:
automoveis = pd.DataFrame.from_records(lista_automoveis)
automoveis

,marca,ano,dono_cpf
0,audi,2021,498.137.502-60
1,citroen,2018,498.137.502-60
2,ford,1987,498.137.502-60
3,audi,2002,405.912.638-15
4,porsche,1979,405.912.638-15
...,...,...,...
58,mercedez,1999,314.650.289-51
59,audi,2007,314.650.289-51
60,mercedez,1977,036.128.974-03
61,peugeut,2002,036.128.974-03


In [10]:
# se a coleção não existir, ela é criada automagicamente :-)
colecao_pessoa = db['pessoas']
pessoa_ids = colecao_pessoa.insert_many(lista_pessoas).inserted_ids
pessoa_ids

[ObjectId('62eee52adeeef0f94766edd7'),
 ObjectId('62eee52adeeef0f94766edd8'),
 ObjectId('62eee52adeeef0f94766edd9'),
 ObjectId('62eee52adeeef0f94766edda'),
 ObjectId('62eee52adeeef0f94766eddb'),
 ObjectId('62eee52adeeef0f94766eddc'),
 ObjectId('62eee52adeeef0f94766eddd'),
 ObjectId('62eee52adeeef0f94766edde'),
 ObjectId('62eee52adeeef0f94766eddf'),
 ObjectId('62eee52adeeef0f94766ede0'),
 ObjectId('62eee52adeeef0f94766ede1'),
 ObjectId('62eee52adeeef0f94766ede2'),
 ObjectId('62eee52adeeef0f94766ede3'),
 ObjectId('62eee52adeeef0f94766ede4'),
 ObjectId('62eee52adeeef0f94766ede5'),
 ObjectId('62eee52adeeef0f94766ede6'),
 ObjectId('62eee52adeeef0f94766ede7'),
 ObjectId('62eee52adeeef0f94766ede8'),
 ObjectId('62eee52adeeef0f94766ede9'),
 ObjectId('62eee52adeeef0f94766edea'),
 ObjectId('62eee52adeeef0f94766edeb'),
 ObjectId('62eee52adeeef0f94766edec'),
 ObjectId('62eee52adeeef0f94766eded'),
 ObjectId('62eee52adeeef0f94766edee'),
 ObjectId('62eee52adeeef0f94766edef'),
 ObjectId('62eee52adeeef0

In [11]:
# se a coleção não existir, ela é criada automagicamente :-)
colecao_automoveis = db['automoveis']
automovel_ids = colecao_automoveis.insert_many(lista_automoveis).inserted_ids
automovel_ids

[ObjectId('62eee52adeeef0f94766edf5'),
 ObjectId('62eee52adeeef0f94766edf6'),
 ObjectId('62eee52adeeef0f94766edf7'),
 ObjectId('62eee52adeeef0f94766edf8'),
 ObjectId('62eee52adeeef0f94766edf9'),
 ObjectId('62eee52adeeef0f94766edfa'),
 ObjectId('62eee52adeeef0f94766edfb'),
 ObjectId('62eee52adeeef0f94766edfc'),
 ObjectId('62eee52adeeef0f94766edfd'),
 ObjectId('62eee52adeeef0f94766edfe'),
 ObjectId('62eee52adeeef0f94766edff'),
 ObjectId('62eee52adeeef0f94766ee00'),
 ObjectId('62eee52adeeef0f94766ee01'),
 ObjectId('62eee52adeeef0f94766ee02'),
 ObjectId('62eee52adeeef0f94766ee03'),
 ObjectId('62eee52adeeef0f94766ee04'),
 ObjectId('62eee52adeeef0f94766ee05'),
 ObjectId('62eee52adeeef0f94766ee06'),
 ObjectId('62eee52adeeef0f94766ee07'),
 ObjectId('62eee52adeeef0f94766ee08'),
 ObjectId('62eee52adeeef0f94766ee09'),
 ObjectId('62eee52adeeef0f94766ee0a'),
 ObjectId('62eee52adeeef0f94766ee0b'),
 ObjectId('62eee52adeeef0f94766ee0c'),
 ObjectId('62eee52adeeef0f94766ee0d'),
 ObjectId('62eee52adeeef0

## Utilizando o aggregate para realizar o join

### Construção do pipeline

In [12]:
sql = """
SELECT *
FROM pessoas
"""

print(sql)


SELECT *
FROM pessoas



In [13]:
sqldf(sql, globals())

,cpf,nome,idade,email
0,498.137.502-60,Ana Clara Moraes,23,anaclara.moraes@gmail.com
1,405.912.638-15,Bruno Vieira,21,bruno.vieira@yahoo.com.br
2,509.634.721-43,Francisco da Cunha,20,francisco.dacunha@ig.com.br
3,738.594.026-92,Marcelo Almeida,24,marcelo.almeida@bol.com.br
4,369.785.041-57,Samuel Nunes,20,samuel.nunes@yahoo.com.br
5,184.930.576-57,Esther Sales,24,esther.sales@yahoo.com.br
6,742.853.691-09,Otávio Pires,18,otávio.pires@yahoo.com.br
7,826.950.173-59,Sophia Costa,14,sophia.costa@gmail.com
8,780.952.341-41,Brenda Nascimento,13,brenda.nascimento@uol.com.br
9,724.560.193-34,Igor da Luz,21,igor.daluz@ig.com.br


In [14]:
sql = """
SELECT *
FROM automoveis
"""

print(sql)


SELECT *
FROM automoveis



In [15]:
sqldf(sql, globals())

,marca,ano,dono_cpf
0,audi,2021,498.137.502-60
1,citroen,2018,498.137.502-60
2,ford,1987,498.137.502-60
3,audi,2002,405.912.638-15
4,porsche,1979,405.912.638-15
...,...,...,...
58,mercedez,1999,314.650.289-51
59,audi,2007,314.650.289-51
60,mercedez,1977,036.128.974-03
61,peugeut,2002,036.128.974-03


### Consulta: 
- quero exibir **pessoas** e os seus **veículos**

In [16]:
sql = """
SELECT *
FROM pessoas as p left join automoveis as a
    on p.cpf = a.dono_cpf
"""

print(sql)


SELECT *
FROM pessoas as p left join automoveis as a
    on p.cpf = a.dono_cpf



In [17]:
sqldf(sql, globals())

,cpf,nome,idade,email,marca,ano,dono_cpf
0,498.137.502-60,Ana Clara Moraes,23,anaclara.moraes@gmail.com,audi,2021.0,498.137.502-60
1,498.137.502-60,Ana Clara Moraes,23,anaclara.moraes@gmail.com,citroen,2018.0,498.137.502-60
2,498.137.502-60,Ana Clara Moraes,23,anaclara.moraes@gmail.com,ford,1987.0,498.137.502-60
3,405.912.638-15,Bruno Vieira,21,bruno.vieira@yahoo.com.br,audi,1998.0,405.912.638-15
4,405.912.638-15,Bruno Vieira,21,bruno.vieira@yahoo.com.br,audi,2002.0,405.912.638-15
...,...,...,...,...,...,...,...
64,314.650.289-51,Ana Beatriz Pinto,28,anabeatriz.pinto@gmail.com,porsche,1984.0,314.650.289-51
65,036.128.974-03,Nathan Barbosa,15,nathan.barbosa@uol.com.br,mercedez,1977.0,036.128.974-03
66,036.128.974-03,Nathan Barbosa,15,nathan.barbosa@uol.com.br,peugeut,2002.0,036.128.974-03
67,036.128.974-03,Nathan Barbosa,15,nathan.barbosa@uol.com.br,suzuki,2019.0,036.128.974-03


In [18]:
pipeline = [
    
    # realiza o left join
    {"$lookup": {
        'from': 'automoveis',
        'localField': 'cpf',
        'foreignField': 'dono_cpf',
        'as': 'automoveis'}
    },
]

pipeline

[{'$lookup': {'from': 'automoveis',
   'localField': 'cpf',
   'foreignField': 'dono_cpf',
   'as': 'automoveis'}}]

In [19]:
pp.pprint(list(colecao_pessoa.aggregate(pipeline)))

[{'_id': ObjectId('62eee52adeeef0f94766edd7'),
  'automoveis': [{'_id': ObjectId('62eee52adeeef0f94766edf5'),
                  'ano': 2021,
                  'dono_cpf': '498.137.502-60',
                  'marca': 'audi'},
                 {'_id': ObjectId('62eee52adeeef0f94766edf6'),
                  'ano': 2018,
                  'dono_cpf': '498.137.502-60',
                  'marca': 'citroen'},
                 {'_id': ObjectId('62eee52adeeef0f94766edf7'),
                  'ano': 1987,
                  'dono_cpf': '498.137.502-60',
                  'marca': 'ford'}],
  'cpf': '498.137.502-60',
  'email': 'anaclara.moraes@gmail.com',
  'idade': 23,
  'nome': 'Ana Clara Moraes'},
 {'_id': ObjectId('62eee52adeeef0f94766edd8'),
  'automoveis': [{'_id': ObjectId('62eee52adeeef0f94766edf8'),
                  'ano': 2002,
                  'dono_cpf': '405.912.638-15',
                  'marca': 'audi'},
                 {'_id': ObjectId('62eee52adeeef0f94766edf9'),
              

### Consulta: 
- quero exibir **automóveis** e os seus respectivos **donos**

In [20]:
sql = """
SELECT *
FROM automoveis as a left join pessoas as p 
    on p.cpf = a.dono_cpf
"""

print(sql)


SELECT *
FROM automoveis as a left join pessoas as p 
    on p.cpf = a.dono_cpf



In [21]:
sqldf(sql, globals())

,marca,ano,dono_cpf,cpf,nome,idade,email
0,audi,2021,498.137.502-60,498.137.502-60,Ana Clara Moraes,23,anaclara.moraes@gmail.com
1,citroen,2018,498.137.502-60,498.137.502-60,Ana Clara Moraes,23,anaclara.moraes@gmail.com
2,ford,1987,498.137.502-60,498.137.502-60,Ana Clara Moraes,23,anaclara.moraes@gmail.com
3,audi,2002,405.912.638-15,405.912.638-15,Bruno Vieira,21,bruno.vieira@yahoo.com.br
4,porsche,1979,405.912.638-15,405.912.638-15,Bruno Vieira,21,bruno.vieira@yahoo.com.br
...,...,...,...,...,...,...,...
58,mercedez,1999,314.650.289-51,314.650.289-51,Ana Beatriz Pinto,28,anabeatriz.pinto@gmail.com
59,audi,2007,314.650.289-51,314.650.289-51,Ana Beatriz Pinto,28,anabeatriz.pinto@gmail.com
60,mercedez,1977,036.128.974-03,036.128.974-03,Nathan Barbosa,15,nathan.barbosa@uol.com.br
61,peugeut,2002,036.128.974-03,036.128.974-03,Nathan Barbosa,15,nathan.barbosa@uol.com.br


In [22]:
pipeline = [
    
    # realiza o left join
    {"$lookup": {
        'from': 'pessoas',
        'localField': 'dono_cpf',
        'foreignField': 'cpf',
        'as': 'dono'}
    },
]

pipeline

[{'$lookup': {'from': 'pessoas',
   'localField': 'dono_cpf',
   'foreignField': 'cpf',
   'as': 'dono'}}]

In [23]:
pp.pprint(list(colecao_automoveis.aggregate(pipeline)))

[{'_id': ObjectId('62eee52adeeef0f94766edf5'),
  'ano': 2021,
  'dono': [{'_id': ObjectId('62eee52adeeef0f94766edd7'),
            'cpf': '498.137.502-60',
            'email': 'anaclara.moraes@gmail.com',
            'idade': 23,
            'nome': 'Ana Clara Moraes'}],
  'dono_cpf': '498.137.502-60',
  'marca': 'audi'},
 {'_id': ObjectId('62eee52adeeef0f94766edf6'),
  'ano': 2018,
  'dono': [{'_id': ObjectId('62eee52adeeef0f94766edd7'),
            'cpf': '498.137.502-60',
            'email': 'anaclara.moraes@gmail.com',
            'idade': 23,
            'nome': 'Ana Clara Moraes'}],
  'dono_cpf': '498.137.502-60',
  'marca': 'citroen'},
 {'_id': ObjectId('62eee52adeeef0f94766edf7'),
  'ano': 1987,
  'dono': [{'_id': ObjectId('62eee52adeeef0f94766edd7'),
            'cpf': '498.137.502-60',
            'email': 'anaclara.moraes@gmail.com',
            'idade': 23,
            'nome': 'Ana Clara Moraes'}],
  'dono_cpf': '498.137.502-60',
  'marca': 'ford'},
 {'_id': ObjectId('62e

### Consulta: 
- quero exibir **pessoas** (nome, idade)
- seus **veículos** (marca, ano)

In [24]:
sql = """
SELECT nome, idade, marca, ano
FROM pessoas as p left join automoveis as a
    on p.cpf = a.dono_cpf
"""

print(sql)


SELECT nome, idade, marca, ano
FROM pessoas as p left join automoveis as a
    on p.cpf = a.dono_cpf



In [25]:
sqldf(sql, globals())

,nome,idade,marca,ano
0,Ana Clara Moraes,23,audi,2021.0
1,Ana Clara Moraes,23,citroen,2018.0
2,Ana Clara Moraes,23,ford,1987.0
3,Bruno Vieira,21,audi,1998.0
4,Bruno Vieira,21,audi,2002.0
...,...,...,...,...
64,Ana Beatriz Pinto,28,porsche,1984.0
65,Nathan Barbosa,15,mercedez,1977.0
66,Nathan Barbosa,15,peugeut,2002.0
67,Nathan Barbosa,15,suzuki,2019.0


In [26]:
pipeline = [
    
    # realiza o left join
    {"$lookup": {
        'from': 'automoveis',
        'localField': 'cpf',
        'foreignField': 'dono_cpf',
        'as': 'automoveis'}
    },
    
    # apenas os campos de interesse
    { "$project": { 
              "nome":1,
              "idade":1,
              'automoveis':{
                    'marca':1,
                    'ano':1
              }
        }
    },
    
]

pipeline

[{'$lookup': {'from': 'automoveis',
   'localField': 'cpf',
   'foreignField': 'dono_cpf',
   'as': 'automoveis'}},
 {'$project': {'nome': 1, 'idade': 1, 'automoveis': {'marca': 1, 'ano': 1}}}]

In [27]:
pp.pprint(list(colecao_pessoa.aggregate(pipeline)))

[{'_id': ObjectId('62eee52adeeef0f94766edd7'),
  'automoveis': [{'ano': 2021, 'marca': 'audi'},
                 {'ano': 2018, 'marca': 'citroen'},
                 {'ano': 1987, 'marca': 'ford'}],
  'idade': 23,
  'nome': 'Ana Clara Moraes'},
 {'_id': ObjectId('62eee52adeeef0f94766edd8'),
  'automoveis': [{'ano': 2002, 'marca': 'audi'},
                 {'ano': 1979, 'marca': 'porsche'},
                 {'ano': 1983, 'marca': 'ford'},
                 {'ano': 1998, 'marca': 'audi'}],
  'idade': 21,
  'nome': 'Bruno Vieira'},
 {'_id': ObjectId('62eee52adeeef0f94766edd9'),
  'automoveis': [{'ano': 2013, 'marca': 'audi'},
                 {'ano': 1979, 'marca': 'mercedez'},
                 {'ano': 1979, 'marca': 'audi'},
                 {'ano': 2003, 'marca': 'ford'}],
  'idade': 20,
  'nome': 'Francisco da Cunha'},
 {'_id': ObjectId('62eee52adeeef0f94766edda'),
  'automoveis': [{'ano': 2018, 'marca': 'peugeut'},
                 {'ano': 1978, 'marca': 'renault'}],
  'idade': 24,
  'n

### Consulta: 
- quero exibir **pessoas** (nome, idade)
- quantos veiculos possuem

In [28]:
sql = """
SELECT p.nome, p.idade, count(a.marca)
FROM pessoas as p left join automoveis as a
    on p.cpf = a.dono_cpf
group by p.nome, p.idade
"""

print(sql)


SELECT p.nome, p.idade, count(a.marca)
FROM pessoas as p left join automoveis as a
    on p.cpf = a.dono_cpf
group by p.nome, p.idade



In [29]:
sqldf(sql, globals())

,nome,idade,count(a.marca)
0,Ana Beatriz Pinto,28,4
1,Ana Clara Moraes,23,3
2,Ana Vitória Dias,19,1
3,Benjamin Cunha,11,0
4,Brenda Nascimento,13,2
5,Bruno Vieira,21,4
6,Bárbara Ribeiro,29,2
7,Davi Lucca Castro,16,0
8,Elisa Aragão,20,0
9,Emilly da Cruz,16,3


In [30]:
pipeline = [
    
    # realiza o left join
    {"$lookup": {
        'from': 'automoveis',
        'localField': 'cpf',
        'foreignField': 'dono_cpf',
        'as': 'automoveis'}
    },
    
    # apenas os campos de interesse
    { "$project": { 
              "nome":1,
              "idade":1,
              'n_automoveis': {'$size': '$automoveis'}
              }
    }
        
]

pipeline



[{'$lookup': {'from': 'automoveis',
   'localField': 'cpf',
   'foreignField': 'dono_cpf',
   'as': 'automoveis'}},
 {'$project': {'nome': 1,
   'idade': 1,
   'n_automoveis': {'$size': '$automoveis'}}}]

In [31]:
pp.pprint(list(colecao_pessoa.aggregate(pipeline)))

[{'_id': ObjectId('62eee52adeeef0f94766edd7'),
  'idade': 23,
  'n_automoveis': 3,
  'nome': 'Ana Clara Moraes'},
 {'_id': ObjectId('62eee52adeeef0f94766edd8'),
  'idade': 21,
  'n_automoveis': 4,
  'nome': 'Bruno Vieira'},
 {'_id': ObjectId('62eee52adeeef0f94766edd9'),
  'idade': 20,
  'n_automoveis': 4,
  'nome': 'Francisco da Cunha'},
 {'_id': ObjectId('62eee52adeeef0f94766edda'),
  'idade': 24,
  'n_automoveis': 2,
  'nome': 'Marcelo Almeida'},
 {'_id': ObjectId('62eee52adeeef0f94766eddb'),
  'idade': 20,
  'n_automoveis': 3,
  'nome': 'Samuel Nunes'},
 {'_id': ObjectId('62eee52adeeef0f94766eddc'),
  'idade': 24,
  'n_automoveis': 3,
  'nome': 'Esther Sales'},
 {'_id': ObjectId('62eee52adeeef0f94766eddd'),
  'idade': 18,
  'n_automoveis': 1,
  'nome': 'Otávio Pires'},
 {'_id': ObjectId('62eee52adeeef0f94766edde'),
  'idade': 14,
  'n_automoveis': 0,
  'nome': 'Sophia Costa'},
 {'_id': ObjectId('62eee52adeeef0f94766eddf'),
  'idade': 13,
  'n_automoveis': 2,
  'nome': 'Brenda Nascim

### Consulta: 
- quero exibir **pessoas** (nome, idade)
- o veículo mais velho que elas possuem

In [32]:
sql = """
SELECT p.nome, p.idade, min(a.ano) as ano
FROM pessoas as p left join automoveis as a
    on p.cpf = a.dono_cpf
group by p.nome, p.idade
"""

print(sql)


SELECT p.nome, p.idade, min(a.ano) as ano
FROM pessoas as p left join automoveis as a
    on p.cpf = a.dono_cpf
group by p.nome, p.idade



In [33]:
sqldf(sql, globals())

,nome,idade,ano
0,Ana Beatriz Pinto,28,1984.0
1,Ana Clara Moraes,23,1987.0
2,Ana Vitória Dias,19,1987.0
3,Benjamin Cunha,11,NaN
4,Brenda Nascimento,13,1994.0
5,Bruno Vieira,21,1979.0
6,Bárbara Ribeiro,29,2003.0
7,Davi Lucca Castro,16,NaN
8,Elisa Aragão,20,NaN
9,Emilly da Cruz,16,1987.0


In [46]:
pipeline = [
    
    # realiza o left join
    {"$lookup": {
        'from': 'automoveis',
        'localField': 'cpf',
        'foreignField': 'dono_cpf',
        'as': 'automoveis'}
    },
    
    # torna a estrutura de veículos "flat"
    {"$unwind":"$automoveis"},
    
    # projeta a nova coluna de ano do veículo
    {"$project":{
        "nome":1,
        "idade":1,
        "ano":"$automoveis.ano"}
    },
    
    # agrupa por nome e idade
    {"$group":{
        '_id': '$_id',
       'nome':{'$first':'$nome'},
       'idade':{'$first':'$idade'},
       'ano': {'$min': '$ano'}    
        }
    }
        
]

pipeline



[{'$lookup': {'from': 'automoveis',
   'localField': 'cpf',
   'foreignField': 'dono_cpf',
   'as': 'automoveis'}},
 {'$unwind': '$automoveis'},
 {'$project': {'nome': 1, 'idade': 1, 'ano': '$automoveis.ano'}},
 {'$group': {'_id': '$_id',
   'nome': {'$first': '$nome'},
   'idade': {'$first': '$idade'},
   'ano': {'$min': '$ano'}}}]

In [47]:
pp.pprint(list(colecao_pessoa.aggregate(pipeline)))

[{'_id': ObjectId('62eee52adeeef0f94766eddb'),
  'ano': 1970,
  'idade': 20,
  'nome': 'Samuel Nunes'},
 {'_id': ObjectId('62eee52adeeef0f94766edeb'),
  'ano': 1970,
  'idade': 22,
  'nome': 'Joaquim Araújo'},
 {'_id': ObjectId('62eee52adeeef0f94766eded'),
  'ano': 2003,
  'idade': 29,
  'nome': 'Bárbara Ribeiro'},
 {'_id': ObjectId('62eee52adeeef0f94766edd7'),
  'ano': 1987,
  'idade': 23,
  'nome': 'Ana Clara Moraes'},
 {'_id': ObjectId('62eee52adeeef0f94766ede5'),
  'ano': 2008,
  'idade': 23,
  'nome': 'Kevin da Costa'},
 {'_id': ObjectId('62eee52adeeef0f94766edd9'),
  'ano': 1979,
  'idade': 20,
  'nome': 'Francisco da Cunha'},
 {'_id': ObjectId('62eee52adeeef0f94766eddf'),
  'ano': 1994,
  'idade': 13,
  'nome': 'Brenda Nascimento'},
 {'_id': ObjectId('62eee52adeeef0f94766ede0'),
  'ano': 1989,
  'idade': 21,
  'nome': 'Igor da Luz'},
 {'_id': ObjectId('62eee52adeeef0f94766ede1'),
  'ano': 1974,
  'idade': 14,
  'nome': 'Isabella Rocha'},
 {'_id': ObjectId('62eee52adeeef0f94766ed